In [ ]:
import plaidml.keras
plaidml.keras.install_backend()

import pickle
import pln_tools_comentarios_vtg as pln
import numpy as np

from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical, np_utils
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
from numpy import array
from pickle import dump
from nltk.tokenize import word_tokenize
from pln_tools_comentarios_vtg import removerSignos




In [ ]:
def getX_y():
    corpus = pickle.load(open("corpus_mtg_afterFilters_allCards.p", "rb"))
    text = [removerSignos(documento, ['{','}','.', ':', '(', ')', ',']) for documento in corpus]

    text = pln.createCorpusText(text) #String con el texto completo del corpus
    text_tokenizado = word_tokenize(text)


    # cut the text in semi-redundant sequences of SEQUENCE_LEN words
    sentences = []
    next_words = []
    for i in range(0, len(text_tokenizado) - SEQUENCE_LEN, STEP):
    sentences.append(text_tokenizado[i: i + SEQUENCE_LEN])
    next_words.append(text_tokenizado[i + SEQUENCE_LEN])


    sentences = [' '.join(sentence) for sentence in sentences]
    corpus_ = corpus[:]
    corpus_.append(' '.join(next_words))
    v = pln.Vectorizer()
    vocab = v.makeVocab(corpus)
    X = v.wordVectorizer(sentences,vocab,SEQUENCE_LEN)
    y = v.wordVectorizer(next_words,vocab,1)
    
    return X,y
    
def produceTexto(numPalabras):
    
    model = load_model("ruta de un model guardado")
    X, _ = getX_y()
    seed_text = np.reshape(X[5000],(1,-1))
    
    print("\n Texto Semilla:")

    print(" ".join([keylist[i] for i in seed_text[0]]))

    texto_output = []

    for i in range(numPalabras):

    yhat = model.predict_classes(seed_text, verbose=0)
    seed_text = np.append(seed_text,yhat[0])
    seed_text = pad_sequences([seed_text], maxlen=SEQUENCE_LEN, truncating='pre')
    texto_output.append(yhat[0])

    print("\n Texto Generado:")
    print(" ".join([keylist[i] for i in texto_output]))
    
    

In [ ]:
X,y = getX_y()
model = Sequential()
model.add(Embedding(len(vocab), 50, input_length=SEQUENCE_LEN))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(vocab), activation='softmax'))
print(model.summary())
Model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=BATCH_SIZE, epochs=1)
model.save('model3.h5')